In [25]:
import pandas as pd
import numpy as np
try:
    import xgboost as xgb
except ImportError:
    %pip install -q xgboost
try:
    from sklearn.metrics import accuracy_score, classification_report
    from sklearn.model_selection import train_test_split
    from sklearn.model_selection import GridSearchCV
    from sklearn.preprocessing import StandardScaler
    from sklearn.impute import SimpleImputer
    from sklearn.mixture import GaussianMixture  # Using GMM for clustering
    from sklearn.manifold import LocallyLinearEmbedding  # For dimensionality reduction
except ImportError:
    %pip install sklearn
try:
    from collections import Counter
except ImportError:
    %pip install collections

In [ ]:
# first, re-create clusters. RF was run on top features predicting clusters
# from Courtney

# data import: ENV
# assumes this script saved one folder up of /data
pfa_data = pd.read_csv('./data/raw_data/PFAS_ENV.csv')

# Display the first few rows of each dataset for exploration
print("PFAs Data:")
print(pfa_data.head())

# Convert 'DATE' column to datetime format
pfa_data['DATE'] = pd.to_datetime(pfa_data['DATE'], format='%m/%d/%Y')

#CLUSTERING MODEL (GMM with 8 Clusters from AIC/BIC)

# Select numeric columns for clustering
numeric_columns = pfa_data.select_dtypes(include=['float64', 'int64']).columns

# Impute missing values with median
imputer = SimpleImputer(strategy='median')
pfa_data_imputed = imputer.fit_transform(pfa_data[numeric_columns])

# Scale the data
scaler = StandardScaler()
pfa_scaled = scaler.fit_transform(pfa_data_imputed)

# Apply Gaussian Mixture Model with the optimal number of clusters (8 clusters)
gmm_final = GaussianMixture(n_components=8, random_state=42)
pfa_data['Cluster'] = gmm_final.fit_predict(pfa_scaled)

PFAs Data:
      NAWQA_ID       DATE  TIME 4_2 FTS-RMK  4_2 FTS-VA 6_2 FTS-RMK  \
0  MIAMSUS1-01  8/21/2019  1200           <         8.0           <   
1  MIAMSUS1-02   7/1/2019  1100           <         7.7           <   
2  MIAMSUS1-03   7/2/2019  1100           <         7.7           <   
3  MIAMSUS1-04   7/9/2019  1100           <         8.0           <   
4  MIAMSUS1-05  7/10/2019  1100           <         8.0           <   

   6_2 FTS-VA 8_2 FTS-RMK  8_2 FTS-VA N-EtFOSAA-RMK  ...  PFPeS-RMK PFPeS-VA  \
0         8.0           <         8.0             <  ...          <      4.0   
1         7.7           <         7.7             <  ...          <      3.8   
2         7.7           <         7.7             <  ...          <      3.8   
3         8.0           <         8.0             <  ...        NaN      7.8   
4         8.0           <         8.0             <  ...          <      4.0   

   PFPeA-RMK PFPeA-VA  PFTeDA-RMK PFTeDA-VA  PFTrDA-RMK PFTrDA-VA  PFUnA-RMK  \
0

In [ ]:
print(np.sort(pfa_data.Cluster.unique()))

[0 1 2 3 4 5 6 7]


In [9]:
key_pfas = ['PFOS-VA', 'PFBA-VA', 'PFHxS-VA', 'PFOA-VA']

In [10]:
# X, y declarations

X = pfa_data[key_pfas]  # Using the top features as the predictors
y = pfa_data['Cluster']  # Cluster labels as the target

# Count instances of each class
class_counts = Counter(y)
print("Class distribution:", class_counts)

# DROP clusters 3, 5, 7 due to sparsity AND 0
valid_classes = [cls for cls, count in class_counts.items() if count >= 5]

# Filter out rows with invalid classes
mask = np.isin(y, valid_classes)
X, y = X[mask], y[mask]

Class distribution: Counter({1: 96, 2: 81, 6: 36, 4: 34, 0: 4, 7: 1, 5: 1, 3: 1})


In [11]:
Counter(y)

Counter({1: 96, 2: 81, 4: 34, 6: 36})

In [ ]:
# stratify y for equal class proportions
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42,stratify=y)

In [22]:
print(Counter(y_train))
print(Counter(y_test))

Counter({1: 67, 2: 56, 6: 25, 4: 24})
Counter({1: 29, 2: 25, 6: 11, 4: 10})


In [23]:
# Re-index the class labels to ensure they start from 0
# Create a mapping of old labels to new ones starting from 0
class_mapping = {label: idx for idx, label in enumerate(np.unique(y_train))}
y_train = np.array([class_mapping[label] for label in y_train])
y_test = np.array([class_mapping[label] for label in y_test])
y_train

array([0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 2, 1, 1, 0, 0, 0, 3, 1, 2, 0,
       0, 1, 0, 1, 1, 0, 2, 3, 1, 1, 1, 2, 0, 0, 0, 2, 0, 1, 0, 2, 0, 1,
       0, 2, 1, 1, 2, 1, 0, 3, 1, 0, 3, 1, 0, 1, 0, 3, 3, 2, 1, 3, 0, 0,
       1, 2, 1, 2, 0, 1, 0, 0, 1, 1, 0, 0, 3, 0, 1, 0, 3, 0, 0, 1, 0, 2,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 2, 1, 1, 0, 1, 2, 0, 1, 0,
       0, 2, 0, 1, 1, 0, 1, 2, 3, 0, 0, 3, 0, 2, 0, 2, 0, 0, 0, 1, 3, 3,
       1, 3, 1, 3, 3, 1, 3, 3, 1, 1, 1, 0, 2, 3, 2, 3, 0, 0, 2, 0, 3, 0,
       1, 0, 1, 1, 2, 0, 1, 0, 3, 3, 0, 2, 1, 1, 0, 2, 3, 3])

## Experiment 1: Predict cluster based off key_pfas levels
This was already done as a baseline using Random Forest. This experiment aims to improve upon this technique by using boosted regression trees. The logic is that the sequential learning nature of this approach could improve accuracy.

In [24]:
# init XBGClassifier
# optimization: softmax
# eval: min log loss

xgb_model = xgb.XGBClassifier(objective="multi:softmax", eval_metric="mlogloss", use_label_encoder=False)
xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.79
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.86      0.86        29
           1       0.79      0.88      0.83        25
           2       0.50      0.50      0.50        10
           3       0.88      0.64      0.74        11

    accuracy                           0.79        75
   macro avg       0.76      0.72      0.73        75
weighted avg       0.79      0.79      0.78        75



In [27]:
# GRID SEARCH for hyperparameter tuning
xgb_clf = xgb.XGBClassifier(objective="multi:softmax", eval_metric="mlogloss", use_label_encoder=False)

# Define the parameter grid
param_grid = {
    'max_depth': [3, 5], # 7
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'gamma': [0, 1, 5],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

grid_search = GridSearchCV(
    estimator=xgb_clf,
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,  # 3-fold cross-validation, limited due to memory constraints
    verbose=2,
    n_jobs=-1  # Use all available cores
)

grid_search.fit(X_train, y_train)

# Best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
Best Parameters: {'colsample_bytree': 0.8, 'gamma': 5, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.8}
Best Accuracy: 0.8662028634805404


It appears that the optimal boosted tree setup includes:
- depth = 3 (max depth of a tree)
- learning_rate = 0.01 (step size, prevent overfitting)
- n_estimators = 200 
- gamma = 5 (min loss reduction for a split)
- subsample = 0.8 (fraction of samples for growing trees)
- colsample_bytree = 0.8 (fraction of features per tree)

Achieving 86.62% accuracy

### TODO: get more metrics or plots showcasing result of experiment

## Experiment 2: Predict amounts of key_pfas based off env/pharma/geo data

This experiment aims to predict concentrations of key PFAS by regressing on the presence or amounts of external features, such as pharmaceutical compunds, geographical features, and other environmental features.


TODO:
- figure out best way to choose features for selection. Maybe running a broad linear regression and evaluating feature importance?
- once features whittled down, join data and extrac X (features) and y (target concentration)
- run train/test split on joined data
- run xgboost baseline + grid search
- plot/summarize results

In [ ]:
# join cols: NAWQA_ID , maybe also DATE and TIME ?

# what we want: get key PFAS from ENV --> join to something like Geospatial, inorganics, pharma and choose some predictors --> 
# --> predict amounts of key PFAS using joined features as predictors. Method of prediction: regression trees